In [ ]:
import numpy as np
import matplotlib.pyplot as plt

rng = np.random.default_rng(11)

def fitness(x):
    return (
        0.12*np.exp(-((x-0.18)/0.08)**2) +
        0.98*np.exp(-((x-0.62)/0.018)**2) +  # narrow peak
        0.10*np.exp(-((x-0.86)/0.06)**2) +
        0.02*np.sin(25*x) + 0.03
    )

def observe(x, noise=0.03):
    return np.clip(fitness(x) + rng.normal(0, noise, size=np.shape(x)), 0, 1)

T = 140
noise = 0.03
init = 15
beta = 2.2
lengthscale = 0.06
grid = np.linspace(0, 1, 1200)

# Shared initialization
x0 = rng.uniform(0, 1, size=init)
y0 = observe(x0, noise=noise)

# Single-shot (random search)
x_ss = np.empty(T); y_ss = np.empty(T)
x_ss[:init] = x0; y_ss[:init] = y0
x_ss[init:] = rng.uniform(0, 1, size=T-init)
y_ss[init:] = observe(x_ss[init:], noise=noise)
best_ss = np.maximum.accumulate(y_ss)

# Closed-loop (UCB on kernel surrogate)
x_cl = np.empty(T); y_cl = np.empty(T)
x_cl[:init] = x0; y_cl[:init] = y0

def kernel(a, b, ls):
    return np.exp(-0.5*((a-b)/ls)**2)

for t in range(init, T):
    X = x_cl[:t]
    Y = y_cl[:t]
    W = kernel(grid[:, None], X[None, :], lengthscale)
    wsum = W.sum(axis=1) + 1e-9
    mu = (W @ Y) / wsum
    density = wsum / wsum.max()
    sigma = 1 - density
    ucb = mu + beta * sigma * 0.25
    x_next = float(grid[np.argmax(ucb)])
    x_cl[t] = x_next
    y_cl[t] = observe(x_next, noise=noise)

best_cl = np.maximum.accumulate(y_cl)

# Plot
plt.figure(figsize=(8.0, 4.8))
plt.plot(best_ss, label='Single-shot (random search)', linewidth=2)
plt.plot(best_cl, label='Closed-loop (UCB refinement)', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Best score observed (max fitness so far)')
plt.title('Toy Comparison (tuned): Closed-loop reaches the narrow optimum faster')
plt.legend()
plt.grid(alpha=0.25)
plt.tight_layout()
plt.show()
